In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/leonardo/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [1]:
import pandas as pd

df = pd.read_csv('../dados/excertos_lematizados.csv', sep = '|')
df.head()

,COD,DESCR_AREA,EXCERTO_LEMATIZADO
0,1400,Responsabilidade,voto : cuidar o auto de tomada de conta especi...
1,1700,Finanças Públicas,voto : cuidar o auto de solicitação de o congr...
2,5700,Responsabilidade,relatório : tratar se de embargo de declaração...
3,284,Direito Processual,voto : XXSECAO em relação a outro processo jud...
4,298,Pessoal,voto : XXSECAO relativo a o ato envolver o sen...


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 

textos = df['EXCERTO_LEMATIZADO'].values
vetorizador = TfidfVectorizer(stop_words = stopwords.words('portuguese'))  #token_pattern=r'\S+', 
vetorizador.fit(textos)
print(len(vetorizador.vocabulary_), 'termos distintos')

26444 termos distintos


In [17]:
matriz = vetorizador.transform(textos)
matriz

<13285x26444 sparse matrix of type '<class 'numpy.float64'>'
	with 2562108 stored elements in Compressed Sparse Row format>

In [18]:
def extrair_tfidf(documento):
    indices = []
    excerto = []
    tf_idf = []
    for termo in textos[documento].split():
        if termo in vetorizador.vocabulary_:
            indice = vetorizador.vocabulary_[termo]
            indices.append(indice)
            excerto.append(termo)
            tf_idf.append(matriz[documento,indice])
    return excerto, indices, tf_idf

In [19]:
excerto, indices, tf_idf = extrair_tfidf(11410)
for i in range(0,21):
    print(excerto[i], '\t', indices[i], '\t', tf_idf[i])

voto 	 25496 	 0.01006802847722078
exame 	 9707 	 0.010621366172999965
solicitação 	 22899 	 0.07561326462310149
congresso 	 5195 	 0.06185230311099829
nacional 	 16531 	 0.05124209925512199
apresentar 	 1597 	 0.04693601576497314
corte 	 5923 	 0.025185609218425904
decisão 	 6580 	 0.009677014536653619
mesa 	 15815 	 0.030675739099701553
diretor 	 7728 	 0.016712028823424662
câmara 	 6362 	 0.05407613576144933
deputado 	 6930 	 0.06358199215642885
termo 	 23989 	 0.030399841028494995
inciso 	 12734 	 0.061060093738559765
artigo 	 1923 	 0.09447054636230383
constituição 	 5358 	 0.05227232856360529
federal 	 10326 	 0.039316052379740965
mediante 	 15685 	 0.03428098941745531
requerer 	 20953 	 0.029706958242720724
envio 	 9090 	 0.022577187206597838
casa 	 3529 	 0.048542592113396905


In [20]:
len(excerto), len(indices), len(tf_idf)

(1082, 1082, 1082)

In [21]:
def extrair_tamanho(documento):
    qtde = 0
    for termo in textos[documento].split():
        if termo in vetorizador.vocabulary_:
            qtde += 1
    return qtde

In [22]:
extrair_tamanho(11410)

1082

In [23]:
for doc in range(0, len(textos)):
    df.loc[doc, 'tamanho'] = extrair_tamanho(doc)
df['tamanho'].describe()

count    13285.000000
mean       316.064057
std        269.888844
min         12.000000
25%        152.000000
50%        240.000000
75%        387.000000
max       2846.000000
Name: tamanho, dtype: float64

In [24]:
import numpy as np

np.percentile(df['tamanho'], 90), np.percentile(df['tamanho'], 95), np.percentile(df['tamanho'], 99)

(614.0, 795.0, 1393.3199999999997)

In [25]:
def filtrar_texto(excerto, tf_idf, maximo):
    derivado = ''
    original = len(tf_idf)
    if maximo >= original:
        minimo = 0
    else:
        minimo = np.percentile(tf_idf,100-100*maximo/original)
    for indice, valor in enumerate(tf_idf):
        if valor > minimo:
            derivado += excerto[indice] + ' '
    return derivado[:-1]

In [39]:
filtrar_texto(excerto, tf_idf, 500)

'tomada conta especial tce instaurar caixa caixa caixa especial quantificar processo artigo instrução normativo tcu caixa quantificar instrução indício dano erário instaurar processo ser quantificar natureza ser ser ser indício dano erário resp resp multa artigo caixa responsável responsável ser tce pressupor constituição processual ser artigo instrução normativo apurar ocorrência dano apurar quantificar dano responsável erário multa artigo ser natureza processo ser representação julgar indício dano ser pressupor constituição tomada conta especial artigo artigo tcu tomada conta especial regularidade indício dano erário responsável multa artigo natureza processo constituição natureza tomada conta especial ser regularidade regularidade dano erário dano ser pressupor processual constituição processo tomada conta especial artigo tcu tomada conta especial apurar responsável quantificar dano julgar tcu responsável natureza dano erário processo ser apurar ocorrência ocorrência ser processo re

In [41]:
for doc in range(0, 5):#len(textos)):
    excerto, _ , tf_idf = extrair_tfidf(doc)
    print('======================================================= Texto', doc, '=======================================================')
    derivado = filtrar_texto(excerto, tf_idf, 500)
    print(derivado, '(' + str(len(derivado.split())) + ')')

======================================================= Texto 0 =======================================================
voto cuidar auto tomada conta especial instaurar gerência executivo instituto nacional seguro social juazeiro norte desfavor exemplo servir senhor condição segurado inss decorrência fraude concessão benefício previdenciários assistir razão unidade técnico jurisprudência corte ser firme sentido ter ser comprovar participação beneficiário concessão regular meio proceder interno apurar adotar autarquia ser possível concluir existência conluio exemplo servir autorizar concessão dever pois ser afastar responsabilidade segurado poder ser mencionar linha recente todo plenário fim linha jurisprudência corte cumprir comunicar instituto nacional seguro social inss procuradoria geral federal pgf inclusão beneficiário presente relação processual impedir adoção providência administrativo judiciais vista reaver valor eventual ir indevido pagos acórdão excluir relação processual sen

In [43]:
for doc in range(0, len(textos)):
    excerto, _ , tf_idf = extrair_tfidf(doc)
    df.loc[doc, 'filtrado'] = filtrar_texto(excerto, tf_idf, 500)
    if (doc + 1) % 1000 == 0:
        print('doc', doc+1, '...')

doc 1000 ...
doc 2000 ...
doc 3000 ...
doc 4000 ...
doc 5000 ...
doc 6000 ...
doc 7000 ...
doc 8000 ...
doc 9000 ...
doc 10000 ...
doc 11000 ...
doc 12000 ...
doc 13000 ...


In [44]:
for doc in range(0, len(textos)):
    df.loc[doc, 'tamanho_derivado'] = len(df.loc[doc, 'filtrado'].split())
df[['tamanho', 'tamanho_derivado']].describe()

,tamanho,tamanho_derivado
count,13285.000000,13285.000000
mean,316.064057,269.965073
std,269.888844,142.944457
min,12.000000,12.000000
25%,152.000000,152.000000
50%,240.000000,240.000000
75%,387.000000,387.000000
max,2846.000000,509.000000


In [45]:
len(df.query('tamanho != tamanho_derivado'))*100/len(df)

15.415882574331953

In [46]:
np.percentile(df['tamanho'], 90), np.percentile(df['tamanho'], 95), np.percentile(df['tamanho'], 99)

(614.0, 795.0, 1393.3199999999997)

In [47]:
np.percentile(df['tamanho_derivado'], 90), np.percentile(df['tamanho_derivado'], 95), np.percentile(df['tamanho_derivado'], 99)

(500.0, 500.0, 500.0)

In [48]:
df.groupby('DESCR_AREA').size().sort_values(ascending = False) * 100 / len(df)

DESCR_AREA
Pessoal                    25.540083
Licitação                  20.745201
Responsabilidade           17.636432
Direito Processual         13.631916
Contrato Administrativo     7.083177
Convênio                    5.141137
Competência do TCU          4.162589
Gestão Administrativa       2.544223
Finanças Públicas           2.468950
Desestatização              1.046293
dtype: float64

In [49]:
df.query('tamanho != tamanho_derivado').groupby('DESCR_AREA').size().sort_values(ascending = False) * 100 / len(df) / (len(df.query('tamanho != tamanho_derivado'))/len(df))

DESCR_AREA
Pessoal                    22.851562
Licitação                  21.728516
Responsabilidade           16.064453
Direito Processual         10.791016
Contrato Administrativo     8.642578
Finanças Públicas           5.566406
Convênio                    5.419922
Competência do TCU          4.541016
Gestão Administrativa       2.343750
Desestatização              2.050781
dtype: float64

introduz um viés...

In [51]:
df.head(10)

,COD,DESCR_AREA,EXCERTO_LEMATIZADO,tamanho,filtrado,tamanho_derivado
0,1400,Responsabilidade,voto : cuidar o auto de tomada de conta especi...,189.0,voto cuidar auto tomada conta especial instaur...,189.0
1,1700,Finanças Públicas,voto : cuidar o auto de solicitação de o congr...,179.0,voto cuidar auto solicitação congresso naciona...,179.0
2,5700,Responsabilidade,relatório : tratar se de embargo de declaração...,106.0,relatório tratar embargo declaração opor exemp...,106.0
3,284,Direito Processual,voto : XXSECAO em relação a outro processo jud...,103.0,voto relação outro processo judiciais tratar r...,103.0
4,298,Pessoal,voto : XXSECAO relativo a o ato envolver o sen...,479.0,voto relativo ato envolver senhor caber rememo...,479.0
5,299,Licitação,voto : XXSECAO o presente apelo ter por objeti...,248.0,voto presente apelo ter objetivo contestar rej...,248.0
6,310,Pessoal,voto : examinar se ato de aposentadoria conced...,364.0,voto examinar ato aposentadoria conceder âmbit...,364.0
7,364,Pessoal,voto : examinar se ato de aposentadoria conced...,136.0,voto examinar ato aposentadoria conceder âmbit...,136.0
8,432,Licitação,voto : tratar se de representação formular por...,536.0,tratar representação formular empresa face pre...,500.0
9,449,Licitação,voto : tratar se de representação formular por...,536.0,tratar representação formular empresa face pre...,500.0


In [52]:
df.to_csv('../dados/excertos_filtrados500.csv', sep = '|', columns = ['COD', 'DESCR_AREA', 'filtrado'], index=False)

In [3]:
import pandas as pd

df = pd.read_csv('../dados/excertos_lematizados.csv', sep = '|')
df.head()

,COD,DESCR_AREA,EXCERTO_LEMATIZADO
0,1400,Responsabilidade,voto : cuidar o auto de tomada de conta especi...
1,1700,Finanças Públicas,voto : cuidar o auto de solicitação de o congr...
2,5700,Responsabilidade,relatório : tratar se de embargo de declaração...
3,284,Direito Processual,voto : XXSECAO em relação a outro processo jud...
4,298,Pessoal,voto : XXSECAO relativo a o ato envolver o sen...


In [4]:
df.loc[11410]

COD                                                               37522
DESCR_AREA                                           Direito Processual
EXCERTO_LEMATIZADO    voto : em exame solicitação de o congresso nac...
Name: 11410, dtype: object

In [5]:
df.loc[11999]

COD                                                               43342
DESCR_AREA                                                      Pessoal
EXCERTO_LEMATIZADO    voto revisor : tratar se de representação ofer...
Name: 11999, dtype: object

In [6]:
df = pd.read_csv('../dados/excertos_filtrados1000.csv', sep = '|')
df.head()

,COD,DESCR_AREA,filtrado
0,1400,Responsabilidade,voto : cuidar o auto de tomada de conta especi...
1,1700,Finanças Públicas,voto : cuidar o auto de solicitação de o congr...
2,5700,Responsabilidade,relatório : tratar se de embargo de declaração...
3,284,Direito Processual,voto : em relação a outro processo judiciais q...
4,298,Pessoal,": a o ato o senhor , rememorar , desde logo , ..."


In [7]:
df.loc[11410]

COD                        37522
DESCR_AREA    Direito Processual
filtrado                     NaN
Name: 11410, dtype: object